In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import requests
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

C:\Users\Sam's PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
path = r"../data/raw/adFontes.csv"
df = pd.read_csv(path)
df['Text'] = ''
df['Swing'] = None

In [3]:
for i in range(df.shape[0]):
    if (df.at[i,'Bias']<=-6):
        df.at[i,'Swing'] = 'Left'
    elif (df.at[i,'Bias']>-6 and df.at[i,'Bias']<6):
        df.at[i,'Swing'] = 'Central'
    elif (df.at[i,'Bias']>=6):
        df.at[i,'Swing'] = 'Right'

In [4]:
df.drop(['Source', 'Bias', 'Quality'], axis=1, inplace=True)

In [5]:
df.columns

Index(['Url', 'Text', 'Swing'], dtype='object')

In [6]:
def Scrape(soup): 
    char = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ']
    ret = []
    body = BeautifulSoup(str(soup.find_all('p')), 'html.parser').text
    body_clean = ''.join(e for e in body if e.lower() in char)
    ret.append(body_clean)

    if (body_clean != '[]' and body_clean != ''):
        return ret
    else:
        return False

In [7]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

[WDM] - Downloading: 100%|██████████| 6.29M/6.29M [00:00<00:00, 13.5MB/s]
C:\Users\Sam's PC\AppData\Local\Temp\ipykernel_10060\1133812237.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
C:\Users\Sam's PC\AppData\Local\Temp\ipykernel_10060\1133812237.py:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


In [8]:
with open (r'../data/ADFONTES/rescrape_ADFONTES.csv', 'w', newline='', encoding='UTF-8') as csvfile:
    fieldnames = df.columns
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()    
    for i in tqdm(range(len(df))):
        try:
            driver.get(df.at[i, 'Url'])
            soup = BeautifulSoup(driver.page_source, 'lxml')
            if soup is not None:
                scrape = Scrape(soup)
                if scrape != False:
                    df.at[i, 'Text'] = scrape
                    writer.writerow({'Url': df.at[i, 'Url'], 'Text': df.at[i, 'Text'], 'Swing': df.at[i, 'Swing']})
                    print('Written Row: ' + str(i))
        except:
            print('No response: ' + str(i))
            continue

  0%|          | 0/1909 [00:00<?, ?it/s]

Written Row: 0
Written Row: 1
Written Row: 2
Written Row: 3
Written Row: 4
Written Row: 5
Written Row: 6
Written Row: 7
Written Row: 8
Written Row: 9
Written Row: 10
Written Row: 11
Written Row: 12
Written Row: 13
Written Row: 14
Written Row: 15
Written Row: 16
Written Row: 17
Written Row: 18
Written Row: 19
Written Row: 20
Written Row: 21
Written Row: 22
Written Row: 23
Written Row: 24
Written Row: 25
Written Row: 26
Written Row: 27
Written Row: 28
Written Row: 29
Written Row: 30
Written Row: 31
Written Row: 32
Written Row: 33
Written Row: 34
Written Row: 35
Written Row: 36
Written Row: 37
Written Row: 38
Written Row: 39
Written Row: 40
Written Row: 41
Written Row: 42
Written Row: 43
Written Row: 44
Written Row: 45
Written Row: 46
Written Row: 47
Written Row: 48
Written Row: 49
Written Row: 50
Written Row: 51
Written Row: 52
Written Row: 53
Written Row: 54
Written Row: 55
Written Row: 56
Written Row: 57
Written Row: 58
Written Row: 59
Written Row: 60
Written Row: 61
Written Row: 62
Wr

In [ ]:
df.head(1900)

,Url,Text,Swing
0,https://abcnews.go.com/Politics/us-disrupted-a...,,Central
1,https://abcnews.go.com/Politics/appeals-court-...,,Central
2,https://abcnews.go.com/Politics/electoral-coll...,,Left
3,https://abcnews.go.com/Politics/facebook-agree...,,Central
4,https://abcnews.go.com/Politics/donald-trump-t...,,Central
...,...,...,...
1895,https://www.wnd.com/2019/02/did-cnn-stack-audi...,,Central
1896,https://www.wnd.com/2019/02/eric-holder-abolis...,,Right
1897,https://www.wnd.com/2019/02/warning-issued-ove...,,Right
1898,https://www.wnd.com/2019/02/guv-moves-to-give-...,,Right


In [ ]:
df.to_csv(r'../data/ADFONTES/rescrape_adfontes.csv')